In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
from prophet import Prophet
from sqlalchemy import create_engine

app = Flask(__name__)

# Initialize Prophet model
m = Prophet(
    changepoint_prior_scale=0.08, 
    seasonality_prior_scale=10.0,
    seasonality_mode='multiplicative',  # Adjust seasonality mode if needed
    yearly_seasonality=True
)

# Define a flag to check if the model has been trained
model_trained = False

@app.route('/train', methods=['POST'])
def train_model():
    # Get database connection details from request
    db_connection_details = request.json['db_connection']

    # Establish database connection
    engine = create_engine(db_connection_details)

    # Query database to retrieve all historical data
    query = "SELECT * FROM your_table"
    df = pd.read_sql(query, con=engine)

    # Prepare dataframe for Prophet
    df['ds'] = pd.to_datetime(df['date_column'])
    df.rename(columns={'quantity_column': 'y'}, inplace=True)

    # Fit the model with all historical data
    m.fit(df)

    # Set the flag to indicate that the model has been trained
    global model_trained
    model_trained = True

    return 'Model trained successfully!'

@app.route('/forecast', methods=['POST'])
def forecast():
    if not model_trained:
        return 'Model has not been trained yet!'

    # Get query parameters from request
    query_parameters = request.json['query_parameters']
    start_date = query_parameters['start_date']
    end_date = query_parameters['end_date']
    forecast_horizon = query_parameters['forecast_horizon']

    # Generate future dates for the forecast horizon
    future = pd.date_range(start=start_date, periods=forecast_horizon, freq='MS')
    future_df = pd.DataFrame({'ds': future})

    # Make forecasts
    forecast = m.predict(future_df)

    # Return forecasts
    return forecast.to_json()

if __name__ == '__main__':
    app.run(debug=True)
